In [1]:
!pip install chromadb langchain duckdb sentence-transformers lark openai pinecone-client
import os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.2/69.2 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 46.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.2/177.2 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.6/922.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 8.6 MB/s eta 0:00:00

In [2]:
from langchain.embeddings import HuggingFaceEmbeddings
model_name = "diptanuc/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}

hf = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


In [5]:

import duckdb
from langchain.document_loaders import DuckDBLoader

loader = DuckDBLoader(
    """SELECT 
    m.title,
    m.description,
    COALESCE(str_split(m.stars,','),list_value('NA')) as stars,
    COALESCE(str_split(m.directors,','),list_value('NA')) as directors,
    m.year,
    COALESCE(m.certificate,'NA') as certificate,
    COALESCE(str_split(m.genre,','),list_value('NA')) as genre,
    COALESCE(m.runtime,'NA') as runtime,
    m.ratingCount,
    m.plot,
    m.summary,
    CAST(m.imdb_rating AS FLOAT) as imdb_rating, 
    m.imdb_title_id as source FROM movie_plots m""",
    database="/content/drive/MyDrive/Colab Notebooks/db.duckdb",
    page_content_columns=["summary", "plot"],
    metadata_columns=[
        "source",
        "title",
        "description",
        "stars",
        "directors",
        "year",
        "certificate",
        "genre",
        "runtime",
        "imdb_rating",
        "ratingCount",
    ],
)

data=loader.load()
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)
print(docs[:4])

[Document(page_content='summary: {{Short description,1965 film by Roman Polanski}}\n{{Use British English,date=November 2021}}\n{{Infobox film\n, name           = Repulsion\n, image          = Repulsion (1965 film poster).jpg\n, caption        = Theatrical release poster\n, director       = Roman Polanski\n, screenplay     = {{plainlist,\n* Roman Polanski\n* Gérard Brach\n* David Stone{{efn,Adaptation and additional dialogue}}\n}}\n, story          = {{plainlist,\n* Roman Polanski\n* Gérard Brach\n}}\n, producer       = Gene Gutowski\n, starring       = {{plainlist,\n* Catherine Deneuve\n* Ian Hendry\n* John Fraser (actor),John Fraser\n* Patrick Wymark\n* Yvonne Furneaux\n}}\n, cinematography = Gilbert Taylor\n, editing        = Alastair McIntyre\n, music          = Chico Hamilton\n, studio         = {{plainlist,\n* Compton Films\n* Tekli British Productions\n}}\n, distributor    = Compton Films\n, released       = {{Film date,df=yes,1965,6,10,United Kingdom}}\n, runtime        = 105 m

In [7]:
from langchain.vectorstores import Pinecone
import pinecone
pinecone.init(api_key=os.environ["PINECONE_API_KEY"], environment=os.environ["PINECONE_ENV"])
# pinecone.create_index("cinemattr", dimension=768)
vectorstore = Pinecone.from_documents(
    docs, hf,index_name="cinemattr"
)

In [ ]:
# from langchain.vectorstores import Chroma
# vectorstore = Chroma.from_documents(
#     data, hf,persist_directory="/content/drive/MyDrive/Colab Notebooks/chromadb"
# )

In [8]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

        
metadata_field_info=[
    AttributeInfo(
        name="title",
        description="The title of the movie", 
        type="string", 
    ),
    AttributeInfo(
        name="description",
        description="The description of the movie", 
        type="string", 
    ),
    AttributeInfo(
        name="genre",
        description="The genre(s) of the movie", 
        type="string or list[string]", 
    ),
    AttributeInfo(
        name="certificate",
        description="The certificate of the movie", 
        type="string", 
    ),
    AttributeInfo(
        name="year",
        description="The year the movie was released", 
        type="integer", 
    ),
    AttributeInfo(
        name="stars",
        description="The name of the movie actors", 
        type="string or list[string]", 
    ),
    AttributeInfo(
        name="director",
        description="The name of the movie directors", 
        type="string or list[string]", 
    ),
    AttributeInfo(
        name="runtime",
        description="The runtime of the movie", 
        type="string", 
    ),
    AttributeInfo(
        name="imdb_rating",
        description="A 1-10 rating for the movie on IMDB",
        type="float"
    ),
     AttributeInfo(
        name="ratingCount",
        description="How many people rated the movie on IMDB. Indicator of movie's popularity",
        type="integer"
    ),
]
document_content_description = "Summary and plot of the movie"
llm = OpenAI(temperature=0.2)
retriever = SelfQueryRetriever.from_llm(llm, vectorstore, document_content_description, metadata_field_info, verbose=True)
retriever.search_kwargs = {"k":10}

In [17]:
retriever.get_relevant_documents("Indian college life")

query='Indian college life' filter=None


[Document(page_content='college.', metadata={'certificate': 'Approved', 'description': 'Four very different college girls drive to Fort Lauderdale, Florida for spring break and seek out various adventures and romance for themselves.', 'directors': ['Henry Levin'], 'genre': ['Comedy', ' Drama', ' Romance'], 'imdb_rating': 6.599999904632568, 'ratingCount': 2870.0, 'runtime': '99 min', 'source': 'tt0054469', 'stars': ['Dolores Hart', 'George Hamilton', 'Yvette Mimieux', 'Jim Hutton'], 'title': 'Where the Boys Are', 'year': 1960.0}),
 Document(page_content='plot:  An American son of Indian immigrants, Krishnagopal Reddy, or "Kris" as he prefers to be called (to distance himself from his heritage), does not associate with the Indian culture that his parents have pushed upon him. Upon arriving at Rutgers University, he finds out to his dismay that all of his roommates are of Indian descent, and all except one are international students from India. His roommates include Ajay (Kal Penn), who i